In [1]:
from datasets import load_dataset

ds = load_dataset("allenai/reward-bench")

d:\Projects\RLHF\rlhf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projects\RLHF\rlhf\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\binit\.cache\huggingface\hub\datasets--allenai--reward-bench. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this articl

In [18]:
ds['raw'].features

{'prompt': Value('string'),
 'chosen': Value('string'),
 'chosen_model': Value('string'),
 'rejected': Value('string'),
 'rejected_model': Value('string'),
 'subset': Value('string'),
 'id': Value('int64')}

In [33]:
ds['raw']

Dataset({
    features: ['prompt', 'chosen', 'chosen_model', 'rejected', 'rejected_model', 'subset', 'id'],
    num_rows: 5123
})

In [26]:
data = [
    {"prompt": "Why did the website crash?",
     "winning": "Too many users logging in caused a memory overflow.",
     "losing": "Aliens disabled the server."},
    {"prompt": "Why did the train stop?",
     "winning": "Signal failure on the main line.",
     "losing": "Because the sun forgot to rise."}
]
data

[{'prompt': 'Why did the website crash?',
  'winning': 'Too many users logging in caused a memory overflow.',
  'losing': 'Aliens disabled the server.'},
 {'prompt': 'Why did the train stop?',
  'winning': 'Signal failure on the main line.',
  'losing': 'Because the sun forgot to rise.'}]

In [20]:
ds['raw']['chosen_model']

Column(['GPT4-Turbo', 'GPT4-Turbo', 'GPT4-Turbo', 'GPT4-Turbo', 'GPT4-Turbo'])

In [27]:
final_dataset = [{"prompt": raw['prompt'], "winning": raw['chosen'], "losing": raw['rejected']} for raw in ds['raw']]

In [ ]:
final_dataset

[{'prompt': 'What are the names of some famous actors that started their careers on Broadway?',
  'winning': 'Several famous actors started their careers on Broadway before making it big in film and television. Here are a few notable examples:\n\n1. Sarah Jessica Parker - Before she was Carrie Bradshaw on "Sex and the City," Sarah Jessica Parker was a Broadway star, having appeared in productions like "Annie" as a child.\n\n2. Meryl Streep - Meryl Streep\'s early career included Broadway productions such as "Trelawny of the \'Wells\'" and "A Memory of Two Mondays / 27 Wagons Full of Cotton."\n\n3. Hugh Jackman - Hugh Jackman won a Tony Award for his role in "The Boy from Oz" and has been known for his stage work as well as his film career.\n\n4. Sutton Foster - Known for her television role in "Younger," Sutton Foster is also a Broadway legend with leading roles in shows like "Thoroughly Modern Millie" and "Anything Goes."\n\n5. Kristen Bell - Before she was the voice of Anna in "Froze

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from src.reward_model import RewardModel, PreferenceDataset, preference_loss
from torch.utils.data import DataLoader
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# # Example synthetic dataset
# data = [
#     {"prompt": "Why did the website crash?",
#      "winning": "Too many users logging in caused a memory overflow.",
#      "losing": "Aliens disabled the server."},
#     {"prompt": "Why did the train stop?",
#      "winning": "Signal failure on the main line.",
#      "losing": "Because the sun forgot to rise."}
# ]

dataset = PreferenceDataset(tokenizer, final_dataset)
dataloader = DataLoader(dataset, batch_size=20, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RewardModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training step
for epoch in range(3):
    for batch in dataloader:
        optimizer.zero_grad()

        winning_scores = model(batch["winning_input_ids"].to(device),
                              batch["winning_mask"].to(device))

        losing_scores = model(batch["losing_input_ids"].to(device),
                                batch["losing_mask"].to(device))

        loss = preference_loss(winning_scores, losing_scores)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")